In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, ConvE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [5]:
model_name = 'convE_transductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = ConvE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameter

Number of parameters: 3893072
Space occupied: 15572288 bytes


In [12]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory transE_transductive already exists.


In [9]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [10]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [11]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [12]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [13]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
#         callbacks="evaluation",
#         callback_kwargs=dict(
#             evaluator=valid_evaluator,
#             evaluation_triples=dataset.validation.mapped_triples,
#             prefix="validation",
#             frequency=1,
#             additional_filter_triples=dataset.training.mapped_triples,
#         ),
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Using RGCN without graph-based sampling! Please select sampler="schlichtkrull" instead of None.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_features=200, out_features=200, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) has parameters, but no reset_parameters.
Layers RGCNLayer(
  (fwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (bwd): BasesDecomposition(
    (relation_representations): LowRankRepresentation(
      (bases): Embedding(
        (_embeddings): Embedding(3, 40000)
      )
    )
  )
  (self_loop): Linear(in_featu

Training epochs on cpu:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/139 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [18]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
          0
0  3.967333


In [19]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cpu:   0%|          | 0.00/35.6k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.022962   0.022962     0.022962
hits_at_1                     0.010968   0.010968     0.010968
hits_at_3                     0.024016   0.024016     0.024016
hits_at_5                     0.030147   0.030147     0.030147
hits_at_10                    0.041002   0.041002     0.041002
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.032993   0.032993     0.032993
hits_at_1                     0.010968   0.010968     0.010968
hits_at_3                     0.037684   0.037684     0.037684
hits_at_5                     0.049101   0.049101     0.049101
hits_at_10                    0.067915   0.067915     0.067915
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.027978   0.027978     0.027978
hits_at_1                     0.010968   0.010968     0.010968
hits_at_3                     0.030850  

In [20]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cpu:   0%|          | 0.00/8.80k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.011896   0.011896     0.011896
hits_at_1                     0.010908   0.010908     0.010908
hits_at_3                     0.011249   0.011249     0.011249
hits_at_5                     0.011590   0.011590     0.011590
hits_at_10                    0.012499   0.012499     0.012499
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.012953   0.012953     0.012953
hits_at_1                     0.010908   0.010908     0.010908
hits_at_3                     0.011817   0.011817     0.011817
hits_at_5                     0.012953   0.012953     0.012953
hits_at_10                    0.015112   0.015112     0.015112
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.012424   0.012424     0.012424
hits_at_1                     0.010908   0.010908     0.010908
hits_at_3                     0.011533  

In [21]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cpu:   0%|          | 0.00/11.2k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.011825   0.011825     0.011825
hits_at_1                     0.010802   0.010802     0.010802
hits_at_3                     0.011069   0.011069     0.011069
hits_at_5                     0.011427   0.011427     0.011427
hits_at_10                    0.012676   0.012676     0.012676
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.012670   0.012670     0.012670
hits_at_1                     0.010712   0.010712     0.010712
hits_at_3                     0.011962   0.011962     0.011962
hits_at_5                     0.012587   0.012587     0.012587
hits_at_10                    0.014372   0.014372     0.014372
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.012247   0.012247     0.012247
hits_at_1                     0.010757   0.010757     0.010757
hits_at_3                     0.011516  

In [22]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [23]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [24]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [25]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                            loss   
15                   embedding_dim   
16                   learning_rate   
17                       optimizer   
18                      num_epochs   
19                        patience   

                                                value  
0                                                 cpu  
1                                            13330400  
2                                

In [26]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)